# 시맨틱 검색 엔진 구축하기
이 노트북은 LangChain의 문서 로더, 임베딩, 그리고 벡터 스토어 추상화 개념에 익숙해지도록 설계되었습니다. 이러한 추상화는 (벡터) 데이터베이스 및 기타 소스에서 데이터를 검색하여 LLM 워크플로우에 통합하도록 지원합니다. 이는 **RAG (Retrieval-Augmented Generation)** 와 같은 모델 추론의 일부로 데이터를 검색해 활용하는 애플리케이션에 중요합니다.

이 노트북에서는 PDF 문서에 대한 검색 엔진을 구축합니다. 이를 통해 입력 쿼리와 유사한 PDF의 특정 구절을 검색할 수 있습니다.

### 개념
이 가이드는 텍스트 데이터 검색에 중점을 둡니다. 아래 개념을 다룰 것입니다:

- 문서 및 문서 로더
- 텍스트 분할기 (Text splitters)
- 임베딩 (Embeddings)
- 벡터 스토어 및 검색기 (Vector stores and retrievers)

### 문서 및 문서 로더 (Documents and Document Loaders)
LangChain은 Document 추상화를 제공합니다. 이 객체는 텍스트 단위와 관련 메타데이터를 나타내기 위해 설계되었습니다.  

Document의 주요 속성:  
- page_content: 문서의 내용을 나타내는 문자열입니다.  
- metadata: 문서의 출처, 다른 문서와의 관계 등 임의의 메타데이터를 포함하는 딕셔너리입니다.  
- id: (선택적) 문서를 식별하는 문자열입니다.  

metadata 속성은 문서의 출처, 문서 간 관계 및 기타 정보를 포함할 수 있습니다. 일반적으로 하나의 Document 객체는 더 큰 문서의 일부(청크)를 나타냅니다.

예제 문서 생성:
```python
from langchain_core.documents import Document

documents = [
    Document(
        page_content="개는 충성심과 친근함으로 잘 알려진 훌륭한 동반자입니다. ",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="고양이는 독립적인 반려동물로, 종종 자신만의 공간을 즐깁니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
```

그러나 LangChain 생태계는 문서 로더(document loaders)를 구현하여 수백 가지 일반적인 소스와 통합할 수 있습니다. 이를 통해 이러한 소스의 데이터를 AI 애플리케이션에 쉽게 통합할 수 있습니다.


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## **문서 로드하기**

PDF 파일을 `Document` 객체의 시퀀스로 로드해 보겠습니다. LangChain 저장소에 [예제 PDF](https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data)가 있습니다. 이 PDF는 **2023년 나이키(Nike)의 10-K 보고서**입니다.  

In [2]:
# pip install pypdf

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/nke-10k-2023_korean.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

103


**`PyPDFLoader`** 는 **PDF의 각 페이지마다 하나의 `Document` 객체**를 로드합니다.  

각 객체에서 다음 정보를 쉽게 접근할 수 있습니다:  

- **페이지의 문자열 콘텐츠**  
- **파일 이름과 페이지 번호를 포함한 메타데이터**  

In [4]:
print(f"{docs[0].page_content[:200]}")
print()
print(docs[0].metadata)

목차 
미국 
증권거래위원회(SEC) 워싱턴 D.C. 20549 
양식 10-K 
(마크 1) 
☑ 2023 년 5 월 31 일에 종료되는 회계 연도에 대한 1934 년 증권거래법 섹션 13 또는 15(D)에 따른 연례 보고서 
또는 
☐ 1934 년 증권거래법(SECURITIES EXCHANGE ACT OF 1934)의 섹션 13 또는 15(D)에 따른 

{'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2024-12-26T10:17:40+09:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2024-12-26T10:17:40+09:00', 'source': './example_data/nke-10k-2023_korean.pdf', 'total_pages': 103, 'page': 0, 'page_label': '1'}


## **텍스트 분할 (Splitting)**  

정보 검색 및 후속 질문-응답(question-answering) 작업을 하는데 **페이지 단위** 사용은 너무 거친 표현일 수 있습니다. 우리의 최종 목표는 **입력 쿼리에 답변할 수 있는 `Document` 객체를 검색하는 것**이므로, PDF를 더 세분화하면 문서의 해당되는 부분이 주변 텍스트에 의해 의미가 흐려지는 것을 방지할 수 있습니다.  

이를 위해 텍스트 분할기(text splitters)를 사용할 수 있습니다. 여기서는 **문자를 기준으로 분할하는 간단한 텍스트 분할기**를 사용할 것입니다.  

- 문서를 **1000자 단위로 분할**합니다.  
- 각 청크(chunk) 사이에는 **200자의 중첩(overlap)** 이 있습니다.  

**중첩**은 중요한 문맥(context)과 문장(statement)이 분리되는 것을 완화하는 데 도움이 됩니다.  

우리는 **RecursiveCharacterTextSplitter**를 사용할 것입니다. 이 분할기는 일반적인 구분자(예: 줄바꿈)를 사용하여 **재귀적으로 문서를 분할**하며, 각 청크가 적절한 크기가 될 때까지 반복합니다. **일반 텍스트 사용 사례에서 권장되는 텍스트 분할기입니다.**  

### **추가 설정**  
- `add_start_index=True`로 설정하면 **각 분할된 `Document`가 원본 `Document` 내에서 시작하는 문자 인덱스**가 **메타데이터 속성(`start_index`)** 으로 보존됩니다.  

In [5]:
# RecursiveCharacterTextSplitter를 가져옵니다.
# 이 도구는 텍스트를 재귀적으로 분할하며, 일반적인 구분자(예: 줄바꿈)를 사용합니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,              # 각 청크(chunk)의 최대 문자 수
    chunk_overlap=200,           # 청크 간 중첩(overlap) 문자 수
    add_start_index=True        # 각 청크의 시작 인덱스를 메타데이터로 추가
)

# 문서를 분할합니다. 'docs'는 분할할 원본 문서 목록입니다.
all_splits = text_splitter.split_documents(docs)

# 분할된 문서의 총 개수
len(all_splits)

283

## **임베딩 (Embeddings)**  

**벡터 검색(Vector Search)** 은 비정형 데이터(예: 비정형 텍스트)를 저장하고 검색하는 일반적인 방법입니다. 핵심 아이디어는 텍스트와 연결된 **숫자 벡터(Numeric Vectors)** 를 저장하는 것입니다.  

쿼리(Query)가 주어지면, 이를 **동일한 차원의 벡터로 임베딩(Embedding)** 하고 **벡터 유사도 측정(metric)** (예: **코사인 유사도**)을 사용하여 관련 텍스트를 식별할 수 있습니다.   

### **모델 선택하기**  

원하는 임베딩 모델을 선택해 사용합니다.  
예를 들어, OpenAI, Hugging Face, Cohere 등 다양한 제공자를 사용할 수 있습니다.  

**임베딩 모델을 선택한 후 벡터 스토어(Vector Store)에 텍스트를 저장하고 검색**할 수 있습니다.

In [6]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

from langchain_huggingface import HuggingFaceEmbeddings

# Hugging Face Hub에서 한국어 임베딩 모델 로드
# KURE-v1은 한국어에 특화된 문장 임베딩 모델입니다.
embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KURE-v1")
embeddings

HuggingFaceEmbeddings(model_name='nlpai-lab/KURE-v1', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
# 쿼리 텍스트를 벡터로 임베딩합니다.
# 각 청크의 내용을 벡터로 변환하여 비교합니다.

# 첫 번째 청크(all_splits[0])의 내용을 벡터로 임베딩합니다.
vector_1 = embeddings.embed_query(all_splits[0].page_content)

# 두 번째 청크(all_splits[1])의 내용을 벡터로 임베딩합니다.
vector_2 = embeddings.embed_query(all_splits[1].page_content)

# 벡터의 길이를 출력합니다.
print(f"생성된 벡터의 길이: {len(vector_1)}\n")

# 첫 번째 벡터의 일부(처음 10개 요소)를 출력합니다.
print(vector_1[:10])

생성된 벡터의 길이: 1024

[-0.03352097421884537, -0.02503204718232155, -0.04472862929105759, 0.04036516323685646, -0.0061622788198292255, 0.024244965985417366, 0.01288260892033577, -0.020397918298840523, 0.0010812670225277543, 0.011760661378502846]


텍스트 임베딩을 생성하는 모델을 준비했다면, 이제 이를 효율적인 **유사도 검색(Similarity Search)** 을 지원하는 **벡터 스토어(Vector Stores)** 에 저장할 수 있습니다.

---

## **벡터 스토어(Vector Stores)**  

LangChain의 [**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 객체는 **텍스트 및 `Document` 객체를 저장**하고, 다양한 **유사도 메트릭(Similarity Metrics)** 을 사용해 쿼리를 수행할 수 있는 메서드를 포함합니다.  

이 객체들은 종종 **임베딩(Embedding)** 모델로 초기화되며, 해당 모델은 **텍스트 데이터를 숫자 벡터로 변환하는 방법**을 결정합니다.   

- 일부 벡터 스토어는 **클라우드 제공업체(Cloud Providers)** 에서 호스팅되며 특정 **자격 증명(Credentials)** 이 필요합니다.  
- 일부 벡터 스토어(예: **Postgres**)는 독립적인 인프라에서 실행되거나 로컬 또는 서드파티 플랫폼을 통해 운영될 수 있습니다.  
- 일부 벡터 스토어는 **인메모리(In-Memory)** 로 실행되어 가벼운 작업에 적합합니다.  

- In-Memory Vector Stroe

In [8]:
%%time
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

ids = vector_store.add_documents(documents=all_splits)
ids[:5]

['18aea082-0b1a-461a-8bcf-b5976ea0979b',
 '1f64db55-7cfd-4503-b2c6-b2c5a00e3a4c',
 '11541bc9-4e6e-4b80-b2f3-6e6486611aa4',
 '0943a947-88eb-4920-bd34-243709a81736',
 '23e8c678-619c-4365-9f3a-6961c5a72ea0']

- Chroma DB

In [13]:
%%time
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

ids = vector_store.add_documents(documents=all_splits)
ids[:5]

['7bd259f7-7492-4e8d-808f-8a8a75f86f94',
 '36ba2cf0-8c43-495a-8120-a110bea0380c',
 'e6e1d5a6-c1be-41ac-a64f-c3caf55deeb3',
 'ea488466-ee4c-4016-8282-212c24e2ec4a',
 '972ec39d-ce41-4a3d-a6af-ee13292bbef9']

## **VectorStore 쿼리하기**  

**`VectorStore`** 가 문서를 포함하고 있는 상태에서, 우리는 이를 쿼리할 수 있습니다.  
[**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)는 다음과 같은 쿼리 메서드를 제공합니다:  

- **동기(Synchronous) 및 비동기(Asynchronous) 방식**  
- **문자열(String) 쿼리 또는 벡터(Vector) 기반 쿼리**  
- **유사도 점수(Similarity Scores) 반환 여부 선택**  
- **유사도(Similarity)** 및 [**최대 주변부 중요도(Maximum Marginal Relevance)**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search)를 사용하여 검색된 결과의 다양성을 균형 있게 조정  

일반적으로 이 메서드들은 출력값으로 [**Document**](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 객체 목록을 포함합니다.  

| **메서드**                     | **입력**           | **출력**                           | **사용 시점**                              |
|--------------------------------|------------------|---------------------------------|--------------------------------------|
| `similarity_search()`          | 텍스트(Query)    | 유사한 문서 리스트                 | 일반적인 유사도 검색                      |
| `similarity_search_with_score()` | 텍스트(Query)    | `(문서, 유사도 점수)` 리스트       | 유사도 점수를 포함하여 검색하고 싶을 때     |
| `similarity_search_by_vector()` | 사전 계산된 벡터 | 유사한 문서 리스트                 | 벡터를 직접 활용한 검색                   |


---

**임베딩(Embeddings)** 은 일반적으로 텍스트를 **'밀집 벡터(Dense Vector)'** 로 표현합니다. 이로 인해 **의미가 유사한 텍스트는 기하학적으로 가까운 위치**에 있게 됩니다.  
이를 통해 **문서에 사용된 특정 키워드를 알지 못해도** 질문을 입력하는 것만으로 관련 정보를 검색할 수 있습니다.  

In [14]:
results = vector_store.similarity_search(
    "Nike는 미국에 몇 개의 유통 센터를 가지고 있습니까?"
)

print(results[0])

page_content='전략을 구현하는 데 중점을 둔 직원들이 사용하고 있습니다. 
미국에는 NIKE 가 8 개의 주요 유통 센터를 보유하고 있습니다. 5 개는 테네시 주 멤피스 또는 그 인근에 위치하고 있으며, 그 중 2 개는 소유이고 3 개는 임대입니다. 
인디애나주 인디애나폴리스와 테네시주 데이턴에 위치한 다른 두 개의 유통 센터는 제 3 자 물류 제공업체가 임대 및 운영합니다. Converse 의 유통 센터 중 하나는 
캘리포니아 온타리오에 있으며 임대되어 있습니다. NIKE 는 미국 이외의 지역에 여러 유통 시설을 보유하고 있으며, 그 중 일부는 제 3 자 물류 제공업체가 임대 및 운영합니다. 
미국 이외의 지역에서 가장 중요한 유통 시설은 벨기에 Laakdal 에 있습니다. 타이창, 중국; 일본 토미사토(Tomisato)와 한국 이천(Icheon)이 모두 소유하고 있습니다. 
에어 매뉴팩처링 이노베이션(Air Manufacturing Innovation)은 오리건주 비버튼(Beaverton) 인근과 베트남 동나이성(Dong Nai Province) 인근에 위치한 나이키 소유 및 임대 
시설과 미주리주 세인트찰스에 있는 나이키 소유 시설에서 신발에 사용되는 쿠션 부품을 제조합니다. 
위에서 설명한 주요 자산 외에도 당사는 판매 및 관리 목적으로 전 세계의 많은 사무실을 임대합니다. 우리는 전 세계적으로 약 1,027 개의 소매점을 임대하고 있으며, 주로 
공장 상점으로 구성되어 있습니다. 소매점에 대한 추가 정보는 "United States Market" 및 "International Markets"를 참조하십시오. 당사의 임대 계약은 2052 회계 연도까지 
다양한 날짜에 만료됩니다. 
항목 3. 법적 절차 
당사는 당사의 사업에 부수적으로 발생하는 일반적인 일상적인 소송을 제외하고는 당사가 당사자이거나 당사의 재산이 대상인 법적 절차가 계류 중이라고 생각하지 않습니다.' metadata={'source': './example_data/nke-10

<br>
- 유사도 점수 (Return scores) 반환

In [15]:
# 'similarity_search_with_score' 메서드는 쿼리와 가장 유사한 문서를 반환합니다.
# 반환값: (문서 객체, 유사도 점수) 쌍의 리스트
results = vector_store.similarity_search_with_score("2023년 Nike의 매출은 얼마였나요?")

# 첫 번째 결과를 추출합니다.
doc, score = results[0]

# 유사도 점수 출력
print(f"Score: {score}\n")

# 검색된 문서 내용 출력
print(doc)

Score: 0.5953285694122314

page_content='목차 
연말 5 월 31 일, 
 
(백만 달러) 2023 2022 2021 
수익      
북아메리카 $ 21,608  $ 18,353 $ 17,179  
유럽, 중동 및 아프리카  13,418   12,479  11,456  
중화권  7,248   7,547  8,290  
아시아 태평양 및 라틴 아메리카  6,431   5,955  5,343  
글로벌 브랜드 사업부  58   102  25  
총 NIKE 브랜드  48,763   44,436  42,293  
대화  2,427   2,346  2,205  
기업  27   (72) 40  
NIKE, INC. 총 매출 $ 51,217  $ 46,710 $ 44,538  
이자 및 세금 차감 전 이익      
북아메리카 $ 5,454  $ 5,114 $ 5,089  
유럽, 중동 및 아프리카  3,531   3,293  2,435  
중화권  2,283   2,365  3,243  
아시아 태평양 및 라틴 아메리카  1,932   1,896  1,530  
글로벌 브랜드 사업부  (4,841)  (4,262) (3,656) 
대화  676   669  543  
기업  (2,840)  (2,219) (2,261) 
이자비용(수입), 순  (6)  205  262  
TOTAL NIKE, INC. 소득세 전 소득 $ 6,201  $ 6,651 $ 6,661  
재산, 공장 및 장비에 대한 추가 사항      
북아메리카 $ 283  $ 146 $ 98  
유럽, 중동 및 아프리카  215   197  153  
중화권  56   78  94  
아시아 태평양 및 라틴 아메리카  64   56  54  
글로벌 브랜드 사업부  271   222  278  
총 NIKE 브랜드  889   699  677  
대화  7   9  7  
기업  140   103  107' metadata={'producer': 'Microsoft® 

----------
내장된 쿼리와의 유사성을 기준으로 문서 반환:

In [16]:
embedding = embeddings.embed_query("2023년 Nike의 마진은 어떻게 영향을 받았나요?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='목차 
 
*도매로 동등 
2023 회계연도 매출 총이익 감소는 주로 다음과 같은 이유 때문이었습니다. 
• 도매 등가 기준으로 NIKE 브랜드 제품 비용 상승, 주로 투입 비용 증가, 인바운드 화물 및 물류 비용 상승, 제품 믹스 때문; 
• NIKE Direct 사업의 마진 감소, 가용 재고 공급 감소로 인한 이전 기간의 낮은 판촉 활동과 비교하여 현재 기간 동안 재고를 청산하기 위한 더 높은 판촉 활동으로 인한 것; 
• 헤지를 포함한 순 외화 환율의 불리한 변화; 그리고 
• 도매와 동등한 기준으로 오프 가격 마진을 낮춥니다. 
이것은 부분적으로 다음과 같이 상쇄되었습니다. 
• 더 높은 NIKE 브랜드 정가 ASP, 도매 등가 기준으로 할인의 순 가치는 주로 전략적 가격 책정 조치 및 제품 믹스로 인합니다. 그리고 
• 주로 2022 회계연도 4 분기에 중화권에서 인식된 더 높은 재고 노후화 준비금으로 인한 기타 비용 감소. 
총 판매 및 관리 비용 
(백만 달러) 2023 회계연도 2022 회계연도 변동률(%) 2021 회계연도 변동률(%) 
수요 창출 비용(1) $ 4,060  $ 3,850   5 % $ 3,114  24 % 
영업간접비  12,317   10,954  12 % 9,911  11 % 
총 판매 및 관리 비용 $ 16,377  $ 14,804   11 % $ 13,025  14 % 
매출 대비 %  32.0 %  31.7 % 30 의 bps 29.2 % 250 의 bps 
(1) 수요 창출 비용은 보증 계약 비용, 무료 제품, TV, 디지털 및 인쇄 광고 및 미디어 비용, 브랜드 이벤트 및 소매 브랜드 프레젠테이션 비용을 포함한 광고 및 판촉 비용으로 구성됩니다. 
2023 회계연도와 2022 회계연도 비교 
2023 회계연도에 수요 창출 비용은 5% 증가했는데, 이는 주로 광고 및 마케팅 비용 증가와 스포츠 마케팅 비용 증가에 기인합니다. 외환 환율 변동은 수요 창출 비용을 약 4% 
포인트 감소시켰습니다

## **검색기 (Retrievers)**  

LangChain의 **`VectorStore` 객체**는 [**Runnable**](https://python.langchain.com/api_reference/core/index.html#langchain-core-runnables)의 서브클래스가 아닙니다. 반면, **[Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers)** 는 **Runnable**의 서브클래스입니다. 따라서 **동기(Synchronous)** 및 **비동기(Asynchronous)** `invoke`와 `batch` 작업과 같은 표준 메서드 집합을 구현합니다.  

### **검색기 vs 벡터 스토어**  
- **벡터 스토어(VectorStore)**: 벡터화된 데이터를 저장하고 쿼리합니다.  
- **검색기(Retriever)**: 벡터 스토어를 활용하거나 외부 API와 같은 **비벡터 데이터 소스**와도 통신할 수 있습니다.  

`@chain`을 이용하여 runnable 형태의 검색기를 직접 구현할 수 있습니다.

In [31]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

# 여러 개의 쿼리를 동시에 실행하여 검색 결과를 반환합니다.
results = retriever.batch(
    [
        "2023년 Nike의 매출은 얼마였나요?",  # 첫 번째 쿼리: 2023년 Nike의 매출 정보
        "2023년 Nike의 마진은 어떻게 영향을 받았나요?",  # 두 번째 쿼리: 2023년 Nike의 마진 영향
    ],
)

results

[[Document(id='b1896093-2fd2-47e2-97fe-dfb44947a0d4', metadata={'total_pages': 103, 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'moddate': '2024-12-26T10:17:40+09:00', 'creationdate': '2024-12-26T10:17:40+09:00', 'source': './example_data/nke-10k-2023_korean.pdf', 'keywords': '0000320187-23-000039; ; 10-K', 'page_label': '83', 'start_index': 0, 'title': '0000320187-23-000039', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'page': 82, 'creator': 'Microsoft® Word Microsoft 365용', 'producer': 'Microsoft® Word Microsoft 365용'}, page_content='목차 \n연말 5 월 31 일, \n \n(백만 달러) 2023 2022 2021 \n수익      \n북아메리카 $ 21,608  $ 18,353 $ 17,179  \n유럽, 중동 및 아프리카  13,418   12,479  11,456  \n중화권  7,248   7,547  8,290  \n아시아 태평양 및 라틴 아메리카  6,431   5,955  5,343  \n글로벌 브랜드 사업부  58   102  25  \n총 NIKE 브랜드  48,763   44,436  42,293  \n대화  2,427   2,346  2,205  \n기업  27   (72) 40  \nNIKE, INC. 총 매출 $ 51,217  $ 46,710 $ 44,538  \n이자 및 세금 차감 전 이익      \n북아메

In [32]:
for doc in results:
    print(doc[0].page_content)

목차 
연말 5 월 31 일, 
 
(백만 달러) 2023 2022 2021 
수익      
북아메리카 $ 21,608  $ 18,353 $ 17,179  
유럽, 중동 및 아프리카  13,418   12,479  11,456  
중화권  7,248   7,547  8,290  
아시아 태평양 및 라틴 아메리카  6,431   5,955  5,343  
글로벌 브랜드 사업부  58   102  25  
총 NIKE 브랜드  48,763   44,436  42,293  
대화  2,427   2,346  2,205  
기업  27   (72) 40  
NIKE, INC. 총 매출 $ 51,217  $ 46,710 $ 44,538  
이자 및 세금 차감 전 이익      
북아메리카 $ 5,454  $ 5,114 $ 5,089  
유럽, 중동 및 아프리카  3,531   3,293  2,435  
중화권  2,283   2,365  3,243  
아시아 태평양 및 라틴 아메리카  1,932   1,896  1,530  
글로벌 브랜드 사업부  (4,841)  (4,262) (3,656) 
대화  676   669  543  
기업  (2,840)  (2,219) (2,261) 
이자비용(수입), 순  (6)  205  262  
TOTAL NIKE, INC. 소득세 전 소득 $ 6,201  $ 6,651 $ 6,661  
재산, 공장 및 장비에 대한 추가 사항      
북아메리카 $ 283  $ 146 $ 98  
유럽, 중동 및 아프리카  215   197  153  
중화권  56   78  94  
아시아 태평양 및 라틴 아메리카  64   56  54  
글로벌 브랜드 사업부  271   222  278  
총 NIKE 브랜드  889   699  677  
대화  7   9  7  
기업  140   103  107
목차 
 
*도매로 동등 
2023 회계연도 매출 총이익 감소는 주로 다음과 같은 이유 때문이었습니다. 
• 도매 등가 기준으로 NIKE

**벡터 스토어(Vectorstores)** 는 **`as_retriever`** 메서드를 구현하며, 이를 통해 **[VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)** 를 생성할 수 있습니다.  

이 검색기(Retriever)는 특정 **`search_type`** 및 **`search_kwargs`** 속성을 포함하며, 이를 통해 **벡터 스토어의 어떤 메서드를 호출할지**, 그리고 **어떻게 매개변수를 설정할지**를 정의합니다.  

| 항목                 | `similarity`                     | `mmr` (Maximum Marginal Relevance)      | `similarity_score_threshold`              |
| ------------------ | -------------------------------- | --------------------------------------- | ----------------------------------------- |
| 🔍 검색 기준           | 질의(query)와 가장 **유사한 벡터**를 단순히 선택 | 유사도 + **결과 간 다양성** 고려                   | 유사도 기준을 **임계값 이상**으로 제한                   |
| 🎯 목적              | 가장 관련성 높은 문서 찾기                  | 중복 없이 **정보 다양성 확보**                     | 너무 유사하지 않은 문서 제거                          |
| ⚙️ 주요 파라미터         | `k`: 반환할 문서 수                    | `k`: 개수<br>`lambda_mult`: 다양성 비율 (0\~1) | `score_threshold`: 최소 유사도<br>`k`: 최대 문서 수 |
| 📈 적합한 경우          | 단순 정보 검색                         | 요약, RAG 등에서 다양한 관점 필요할 때                | 정확도 높은 문서만 사용하고 싶을 때                      |
| 🚫 주의점             | 중복된 내용이 나올 수 있음                  | 다양성 높이려다 **정확도 저하** 가능                  | `score_threshold` 너무 높으면 결과 없을 수도 있음      |


예를 들어, 위의 예제를 다음과 같이 재현할 수 있습니다:  

In [36]:
# `as_retriever` 메서드를 사용하여 벡터 스토어를 검색기 객체로 변환합니다.
retriever = vector_store.as_retriever(
    search_type="similarity",  # 검색 유형을 'similarity'(유사도)로 설정
    search_kwargs={"k": 1},  # 상위 1개의 유사한 결과만 반환하도록 설정
)

# MMR 방식 (Maximum Marginal Relevance) - 검색 결과 다양성을 증가
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "lambda_mult": 0.5}  # lambda_mult 조절 가능 (0~1 사이 값)
)

# Similarity Score Threshold 방식 - 일정 유사도 이상인 문서만 반환
retriever_threshold = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8, "k": 1}  # 0.8 이상의 유사도 점수를 가진 문서만 반환
)

# 여러 개의 쿼리를 동시에 실행하여 검색 결과를 반환합니다.
retriever.batch(
    [
        "2023년 Nike의 매출은 얼마였나요?",  # 첫 번째 쿼리: 2023년 Nike의 매출 정보
        "2023년 Nike의 마진은 어떻게 영향을 받았나요?",  # 두 번째 쿼리: 2023년 Nike의 마진 영향
    ],
)

[[Document(id='b1896093-2fd2-47e2-97fe-dfb44947a0d4', metadata={'start_index': 0, 'title': '0000320187-23-000039', 'creationdate': '2024-12-26T10:17:40+09:00', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'moddate': '2024-12-26T10:17:40+09:00', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'creator': 'Microsoft® Word Microsoft 365용', 'page_label': '83', 'total_pages': 103, 'page': 82, 'producer': 'Microsoft® Word Microsoft 365용', 'keywords': '0000320187-23-000039; ; 10-K', 'source': './example_data/nke-10k-2023_korean.pdf'}, page_content='목차 \n연말 5 월 31 일, \n \n(백만 달러) 2023 2022 2021 \n수익      \n북아메리카 $ 21,608  $ 18,353 $ 17,179  \n유럽, 중동 및 아프리카  13,418   12,479  11,456  \n중화권  7,248   7,547  8,290  \n아시아 태평양 및 라틴 아메리카  6,431   5,955  5,343  \n글로벌 브랜드 사업부  58   102  25  \n총 NIKE 브랜드  48,763   44,436  42,293  \n대화  2,427   2,346  2,205  \n기업  27   (72) 40  \nNIKE, INC. 총 매출 $ 51,217  $ 46,710 $ 44,538  \n이자 및 세금 차감 전 이익      \n북아메